In [39]:
print(1)

1


In [1]:
import pandas as pd
import os
import numpy as np
from dateExt import dateExtractor

dx = dateExtractor()

In [12]:
RES_DIR = os.path.join("Results", "5 - UpScale")
res = pd.read_csv(os.path.join(RES_DIR, "Normal Output.csv"))
res.drop("Unnamed: 0", axis=1, inplace=True)
res.head()

,0,1
0,9347d5f7.jpeg,NaN
1,459b00d4.jpeg,NaN
2,0a8a955f.jpeg,NaN
3,9f6518d2.jpeg,2019-02-24
4,fc4067f4.jpeg,NaN


In [13]:
y = pd.read_excel("meta data.xlsx")
y.drop(["Unnamed: 0", "Image Remarks", "Models output (Raw)", "Model Output (Processed)"], axis = 1, inplace=True)
y[1]  = y[1].astype(str)
y.drop(y[y[1]=="nan"].index, axis = 0, inplace=True)
print(len(y))
y.loc[y[1]=="---"] = "NaN"

158


In [14]:
for i, dt in enumerate(y[1]):
    try:
        y['Expected Output'][i] = dx.extractDate(dt)
    except Exception as e:
        print(str(e))

{'sep'}
{'jun'}
{'june'}
{'aug'}
{'jan'}
{'jun'}
list index out of range
{'december'}
{'jul'}


In [15]:
y.rename(columns = {0: "ID", 1: "Actual Date"}, inplace=True)
res.rename(columns= {'0': "ID", '1':"Hypo"}, inplace=True)

In [16]:
y.head()

,ID,Actual Date,Expected Output
0,0a0ebd53.jpeg,29-MAY-2019,2019-05-29
1,0a8a955f.jpeg,22.05.2019,2019-05-22
2,0b1664cb.jpeg,05/07/2019,2019-07-05
3,0c60aa84.jpeg,28-Jun-2019,2019-06-28
4,0c4672b4.jpeg,18/06/2019,2019-06-18


In [17]:
res.head()

,ID,Hypo
0,9347d5f7.jpeg,NaN
1,459b00d4.jpeg,NaN
2,0a8a955f.jpeg,NaN
3,9f6518d2.jpeg,2019-02-24
4,fc4067f4.jpeg,NaN


In [18]:
s = y.merge(res, how = 'left') #, left_index=True, right_index=True)
s

,ID,Actual Date,Expected Output,Hypo
0,0a0ebd53.jpeg,29-MAY-2019,2019-05-29,NaN
1,0a8a955f.jpeg,22.05.2019,2019-05-22,NaN
2,0b1664cb.jpeg,05/07/2019,2019-07-05,NaN
3,0c60aa84.jpeg,28-Jun-2019,2019-06-28,2019-06-28
4,0c4672b4.jpeg,18/06/2019,2019-06-18,NaN
...,...,...,...,...
153,f9034671.jpeg,06/17/2019,2019-06-17,NaN
154,f2e9be84.jpeg,22/05/2019,2019-05-22,2019-05-04
155,fb360175.jpeg,25AUG’17,2017-08-25,NaN
156,fb939620.jpeg,06/16/2019,2019-06-16,NaN


In [19]:
s.loc[s['Hypo']=="null", 'Hypo'] = "NaN"

In [20]:
print("Accuracy is: ", len(s.loc[s['Expected Output']==s['Hypo']])/len(s)*100)

Accuracy is:  32.91139240506329


## Results: 

|Test No: |Model            |Accuracy |
|---------|-----------------|---------|
|2        |W/o Auto correct |29.746835|
|3        |W Auto Correct   |30.379747|
|4        |W pre processing |4.4303794|
|5        |W UpScaling only |32.911392|

## Some minor Analysis

In [58]:
s.head(50)
s.loc[s['Expected Output']!=s['Hypo']]

,ID,Actual Date,Expected Output,Hypo
0,0a0ebd53.jpeg,29-MAY-2019,2019-05-29,NaN
1,0a8a955f.jpeg,22.05.2019,2019-05-22,NaN
2,0b1664cb.jpeg,05/07/2019,2019-07-05,2019-05-87
4,0c4672b4.jpeg,18/06/2019,2019-06-18,NaN
5,0cc46a10.jpeg,03/Jun/2019,2019-06-03,2004-08-09
...,...,...,...,...
152,f980a8bc.jpeg,29/05/2019,2019-05-29,NaN
153,f9034671.jpeg,06/17/2019,2019-06-17,NaN
155,fb360175.jpeg,25AUG’17,2017-08-25,NaN
156,fb939620.jpeg,06/16/2019,2019-06-16,NaN
